In [141]:
a = ['2', '4']
''.join(a)

'2\n4'

In [138]:
a = np.arange(10)
a = np.reshape(a, (2, 5))
str(a.sum())

'45'

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv3D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense, LSTM, GRU, BatchNormalization, LayerNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd


filePath = './trainData/small_image/small_image_train'

file = os.listdir(filePath)

x, y = [], []
for fi in file:
    im = Image.open(filePath + '/' + fi)
    img = np.array(im)
    
    x.append(img)
    y.append(fi)
    

data = pd.read_csv('./goodData/amos0313_last.csv')
def getlabel(date):
#     比如030200，代表3点2分
    hour = int(date[:2])
    minute = int(date[2:4])
    
    idNum = hour * 60 + minute + 959
#     print(idNum)
    try:
        label = data.at[idNum, 'MOR']
#         print(label)
    except:
        print(hour, minute, idNum)
        
    return label

# 这里的trianx和y 没有按照时间顺序来

trainY = np.array([getlabel(time[:6]) for time in y], dtype=np.float32)

trainX = np.array(x, dtype = np.float32) / 255

# trainX.dtype = np.float32

class resNet50(Model):
    def __init__(self):
        super(resNet50, self).__init__()
        self.res50 = tf.keras.applications.ResNet50(include_top=False, input_shape=(72, 128, 3) )
        self.f1 = Flatten()
        self.d1 = Dense(512, activation='relu')
#         self.d2 = Dense(1, activation='linear')
    
    def call(self, x):
        y = self.res50(x)
        y = self.f1(y)
        y = self.d1(y)
#         y = self.d2(y)
        return y

class visionModel(Model):
    def __init__(self):
        super(videoVisionModel, self).__init__()
        
        # CNN input: batch * height * width * channels
        # CNN output: batch * 512(features)
        self.cnn1 = resNet50()
        self.cnn2 = resNet50()
        
#         self.cnn3d1 = Conv3D
        
        # GRU input: batch * seq * feature(512)
        # 上面CNN output 需要在第二个维度 叠加输入给 GRU
        self.gru = GRU(rnn_units, return_sequences=False, stateful=True, recurrent_initializer='glorot_uniform')

        # GRU output: if return_sequences=False : batch * rnnunits
        # GRU output: if return_sequences=True : batch * seqLen * rnnunits
        
        # 最终output 每个batch需要一个值，对应最后一张图片的能见度
        
        self.f1 = Flatten()
        
        self.d1 = Dense(512, activation='relu')
        self.d2 = Dense(1, activation='linear')
        
    def call(self, x):
        #inputX : batch * picNum * height * width * channels
        
        pic0 = x[:, 0, :, :, :]
        pic1 = x[:, 1, :, :, :]
#         pic2 = x[:, 2, :, :, :]

        fea0 = self.cnn1(pic0)
        fea1 = self.cnn1(pic1)
        
        # batch * seqlen * feat
        feat = tf.stack([feat0, feat1], axis=1)
        y = self.gru(feat)
        # todo :
        return y
        
model = videoVisionModel()
model.compile(optimizer='adam', 
              loss = 'mean_squared_error'
            )


# history = model.fit(x=trainX, y=trainY, batch_size=16, epochs=30)



In [59]:
pic0 = trainX[0]
pic1 = trainX[1]

pic = tf.stack([pic0, pic1], axis=0)
pic = tf.reshape(pic, [1, 2, 72, 128, 3])
pic.shape


TensorShape([1, 2, 72, 128, 3])

In [47]:
input_shape[1:]

(28, 28, 1)

In [66]:
input_shape =(4, 28, 28, 28, 1)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv3D(10, 2, activation='relu', input_shape=pic[1:], padding='same')(pic)
print(y.shape)

input_shape[1:]




(1, 2, 72, 128, 10)


(28, 28, 28, 1)

In [118]:

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv3D, BatchNormalization, Activation, MaxPool2D, MaxPool3D, Dropout, Flatten, Dense, LSTM, GRU, BatchNormalization, LayerNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

frameLen = 5
class visionModel(Model):
    def __init__(self):
        # 不指定inputshape仍然work
        super(visionModel, self).__init__()
        self.c1 = Conv3D(filters=64, kernel_size=(2,5,5), strides=(1, 1, 1), input_shape=(frameLen, 72, 128, 3), padding='same')  # 卷积层
        # self.c1 = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')

        self.c2 = Conv3D(filters=64, kernel_size=(2,5,5), strides=(1,1,1), padding='same')  # 卷积层
        # self.c2 = Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same')  # 卷积层
        self.b2 = BatchNormalization()
        self.a2 = Activation('relu')

        self.p1 = MaxPool3D(pool_size=(2, 3, 3), strides=(2, 2, 2), padding='same')  # 池化层
        # self.p1 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')  # 池化层
        self.d1 = Dropout(0.2)

        self.c3 = Conv3D(filters=128, kernel_size=(2,5,5), padding='same')  # 卷积层
        # self.c3 = Conv2D(filters=128, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b3 = BatchNormalization()
        self.a3 = Activation('relu')

        self.c4 = Conv3D(filters=128, kernel_size=(2,5,5), padding='same')  # 卷积层
        self.b4 = BatchNormalization()
        self.a4 = Activation('relu')

        self.p2 = MaxPool3D(pool_size=(2, 3, 3), strides=(2, 2, 2), padding='same')  # 池化层
        self.d2 = Dropout(0.2)

        self.c5 = Conv3D(filters=256, kernel_size=(2,5,5), padding='same')  # 卷积层
        # self.c5 = Conv2D(filters=256, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b5 = BatchNormalization()
        self.a5 = Activation('relu')

        # 先搞5层试试


        self.c6 = Conv2D(filters=256, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b6 = BatchNormalization()
        self.a6 = Activation('relu')

        self.c7 = Conv2D(filters=256, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b7 = BatchNormalization()
        self.a7 = Activation('relu')

        self.p3 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')  # 池化层
        self.d3 = Dropout(0.2)

        self.c8 = Conv2D(filters=512, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b8 = BatchNormalization()
        self.a8 = Activation('relu')

        self.c9 = Conv2D(filters=512, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b9 = BatchNormalization()
        self.a9 = Activation('relu')

        self.c10 = Conv2D(filters=512, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b10 = BatchNormalization()
        self.a10 = Activation('relu')

        self.p4 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')  # 池化层
        self.d4 = Dropout(0.2)

        self.c11 = Conv2D(filters=512, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b11 = BatchNormalization()
        self.a11 = Activation('relu')

        self.c12 = Conv2D(filters=512, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b12 = BatchNormalization()
        self.a12 = Activation('relu')

        self.c13 = Conv2D(filters=512, kernel_size=(3, 3), padding='same', strides=1)  # 卷积层
        self.b13 = BatchNormalization()
        self.a13 = Activation('relu')

        self.p5 = MaxPool2D(pool_size=(2, 2), strides=2, padding='same')  # 池化层
        self.d5 = Dropout(0.2)


        self.gru1 = GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')
        self.layerNor1 = LayerNormalization()
        self.drop1 = Dropout(0.2)
        self.gru2 = GRU(512, return_sequences=False, stateful=True, recurrent_initializer='glorot_uniform')

        self.dense1 = Dense(256, activation='relu')
        self.dense2 = Dense(1, activation='linear')

        self.flatten = Flatten()
        self.f1 = Dense(512, activation='relu')
        self.d1 = Dropout(0.2)
        self.f2 = Dense(512, activation='relu')
        self.d2 = Dropout(0.2)
        self.f3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.a1(self.b1(self.c1(x)))
        x = self.a2(self.b2(self.c2(x)))
        x = self.d1(self.p1(x))

        x = self.a3(self.b3(self.c3(x)))
        x = self.a4(self.b4(self.c4(x)))
        x = self.d2(self.p2(x))

        x = self.a5(self.b5(self.c5(x)))

        # x = self.a6(self.b6(self.c6(x)))
        # x = self.a7(self.b7(self.c7(x)))
        # x = self.d3(self.p3(x))

        # x = self.a8(self.b8(self.c8(x)))
        # x = self.a9(self.b9(self.c9(x)))
        # x = self.a10(self.b10(self.c10(x)))
        # x = self.d4(self.p4(x))

        # x = self.a11(self.b11(self.c11(x)))
        # x = self.a12(self.b12(self.c12(x)))
        # x = self.a13(self.b13(self.c13(x)))
        # x = self.d5(self.p5(x))
        x = tf.reshape(x, [x.shape[0], x.shape[1], -1])
        
        x = self.gru2(self.drop1(self.layerNor1(self.gru1(x))))
        y = self.dense2(self.dense1(x))
        # y = self.f3(self.d2(self.f2(self.d1(self.f1(self.flatten(x))))))
        return y


In [133]:
model = visionModel()
pic0 = trainX[0]
pic1 = trainX[1]

pic = tf.stack([pic0, pic1, pic0, pic0, pic1], axis=0)
pic = tf.reshape(pic, [1, 5, 72, 128, 3])

y = np.array([[1]])

model(pic).shape
model.compile(optimizer='adam', loss = 'mean_squared_error')

history = model.fit(x=pic, y=y, batch_size=1, epochs=30)

Epoch 1/30
1/1 [==============================] - 0s 14ms/step - loss: 1.9400
Epoch 2/30


KeyboardInterrupt: 

In [131]:
data = data.squeeze()
data.shape
pic.shape

TensorShape([1, 5, 72, 128, 3])

In [128]:

y = trainY[:4]
y

array([ 1., 18.,  1.,  1.], dtype=float32)

In [130]:
model.compile(optimizer='adam', loss = 'mean_squared_error')

history = model.fit(x=data, y=y, batch_size=16, epochs=30)

Epoch 1/30


TypeError: in user code:

    /Users/zmystery/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-118-733d85a94fde>:131 call  *
        x = tf.reshape(x, [x.shape[0], x.shape[1], -1])
    /Users/zmystery/anaconda3/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /Users/zmystery/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:195 reshape
        result = gen_array_ops.reshape(tensor, shape, name)
    /Users/zmystery/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gen_array_ops.py:8234 reshape
        "Reshape", tensor=tensor, shape=shape, name=name)
    /Users/zmystery/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:473 _apply_op_helper
        raise err
    /Users/zmystery/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

In [109]:

class Cifar10_Model(Model):
    def __init__(self):
        super(Cifar10_Model, self).__init__()
        
        self.c1 = tf.keras.layers.Conv3D(input_shape=(2, 72, 128, 3), filters=32, kernel_size=(1,3,3), strides=(1, 1, 1), padding='valid')  # 卷积层
        self.b1 = BatchNormalization()  # BN层
        self.a1 = Activation('relu')  # 激活层
        
        self.m1 = tf.keras.layers.MaxPool3D(pool_size=(1, 2, 2), padding='valid')  # 池化层
        self.drop1=Dropout(0.2)  # dropout层
        
        self.c2 = Conv2D(64, kernel_size=(5,5), padding='same')
        self.b2 = BatchNormalization()
        self.a2 = Activation('relu')
        self.m2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2, padding='same')
        self.drop2 = Dropout(0.2)

        self.flatten = Flatten()
        self.d1 = Dense(512, activation='relu')
        self.drop3 = Dropout(0.2)
        self.d2 = Dense(10, activation='softmax')
        
        self.gru1 = GRU(512, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')
        self.gru2 = GRU(256, return_sequences=False, stateful=True, recurrent_initializer='glorot_uniform')

        # GRU output: if return_sequences=False : batch * rnnunits
        # GRU output: if return_sequences=True : batch * seqLen * rnnunits
        
        # 最终output 每个batch需要一个值，对应最后一张图片的能见度
        
        self.f1 = Flatten()
        
        self.d1 = Dense(512, activation='relu')
        self.d2 = Dense(1, activation='linear')
        

    def call(self, x):
        x=self.c1(x)
        x=self.b1(x)
        x=self.a1(x)
        x=self.m1(x)
        x=self.drop1(x)
        
        y = tf.reshape(x, [x.shape[0], x.shape[1], -1])
        
        y = self.gru1(y)
        y = self.gru2(y)

#         y = self.d1(y)
#         y = self.d2(y)
#         x=self.c2(x)
#         x=self.b2(x)
#         x=self.a2(x)
#         x=self.m2(x)
#         x=self.drop2(x)

#         x=self.flatten(x)
#         x=self.d1(x)
#         x=self.drop3(x)
#         y=self.d2(x)
        return y


In [110]:
model = Cifar10_Model()
model(pic).shape

TensorShape([1, 256])

In [78]:
Flatten()

In [26]:
feat1 = tf.Variable([[1, 2],[3, 4], [5, 6]])
feat0 = tf.Variable([[11, 22],[33, 44], [55, 66]] )

tf.stack([feat0,feat1],axis=1)

<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[11, 22],
        [ 1,  2]],

       [[33, 44],
        [ 3,  4]],

       [[55, 66],
        [ 5,  6]]], dtype=int32)>

In [3]:
trainX.shape, trainY.shape, trainY.dtype, trainX.dtype

((466, 720, 1280, 3), (466,), dtype('float32'), dtype('float32'))